# Principal Component Analysis

In [2]:
# Import dependencies
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import hvplot.pandas

In [4]:
# Loading data
file_path = "new_iris_data.csv"
df_iris = pd.read_csv(file_path)
df_iris.head()

,sepal_length,petal_length,sepal_width,petal_width
0,5.1,1.4,3.5,0.2
1,4.9,1.4,3.0,0.2
2,4.7,1.3,3.2,0.2
3,4.6,1.5,3.1,0.2
4,5.0,1.4,3.6,0.2


- There are four features in this dataset with values on different scales. 
- The first step in PCA is to standardize these features by using the StandardScaler library

In [6]:
# Standarized data with StandarScaler
iris_scaled = StandardScaler().fit_transform(df_iris)
print(iris_scaled[0:5])

[[-0.90068117 -1.3412724   1.03205722 -1.31297673]
 [-1.14301691 -1.3412724  -0.1249576  -1.31297673]
 [-1.38535265 -1.39813811  0.33784833 -1.31297673]
 [-1.50652052 -1.2844067   0.10644536 -1.31297673]
 [-1.02184904 -1.3412724   1.26346019 -1.31297673]]


- Now that the data has been standardized, we can use PCA to reduce the number of features. 
- The PCA method takes an argument of n_components, which will pass in the value of 2, thus reducing the features from 4 to 2

In [7]:
# Initialize PCA model
pca = PCA(n_components=2)

- After creating the PCA model, we apply dimensionality reduction on the scaled dataset:

In [8]:
# Get two principal components for the iris data.
iris_pca = pca.fit_transform(iris_scaled)

- After this dimensionality reduction, we get a smaller set of dimensions called principal components. These new components are just the two main dimensions of variation that contain most of the information in the original dataset.

- The resulting principal components are transformed into a DataFrame to fit K-means

In [18]:
# Transform PCA data ti a DataFrame
df_iris_pca= pd.DataFrame(data= iris_pca)
df_iris_pca.head()

,0,1
0,-2.264542,0.505704
1,-2.086426,-0.655405
2,-2.367950,-0.318477
3,-2.304197,-0.575368
4,-2.388777,0.674767


- column 0 = principal component 1
- column 1= principal component 2

In [19]:
#Use explained_variance_ratio to learn how much information can be attributed to each principal component
#Fetch the explained variance
pca.explained_variance_ratio_

array([0.72770452, 0.23030523])

### What this tells us?
- the first principal component contains 72.77% of the variance
- the second contains 23.03%. 
- Together, they contain 95.80% of the information.

### Use the elbow curve with the generated principal components and see the K value is 3

In [20]:
# Find the best value for K
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of K values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(df_iris_pca)
    inertia.append(km.inertia_)

# Create the elbow curve
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

C:\Users\alelo\anaconda3\envs\mlenv\lib\site-packages\sklearn\cluster\_kmeans.py:1037: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  "KMeans is known to have a memory leak on Windows "


:Curve   [k]   (inertia)

Use the principal components data with the K-means algorithm with a K value of 3. We could consider 2, but the direction shifts more after 3

In [21]:
# Initialize the K-means model
model = KMeans(n_clusters=3, random_state=0)

# Fit the model
model.fit(df_iris_pca)

# Predict clusters
predictions = model.predict(df_iris_pca)

# Add the predicted class columns
df_iris_pca["class"] = model.labels_
df_iris_pca.head()

,0,1,class
0,-2.264542,0.505704,0
1,-2.086426,-0.655405,0
2,-2.367950,-0.318477,0
3,-2.304197,-0.575368,0
4,-2.388777,0.674767,0


### Finally, we can plot the clusters. Instead of a 3D plot, the data is easier to analyze with only two features

In [23]:
df_iris_pca.hvplot.scatter(
    x="0",
    y="1",
    hover_cols=["class"],
    by="class",
)

:NdOverlay   [class]
   :Scatter   [0]   (1,class)